In [1]:
from directory_tree import display_tree

display_tree('../../MachineLearningStudy')

MachineLearningStudy/
├── datasets/
│   ├── bank-4.zip
│   └── car-4.zip
├── DecisionTree/
│   ├── __init__.py
│   ├── dev.ipynb
│   └── run.py
├── LICENSE
├── main.py
└── README.md


In [2]:
'''
3. read the training data (list or a pandas dataframe)
4. explore the data (e.g. print the first few rows, check the shape, check the columns, check the data types)
'''

import zipfile
import pandas as pd
import io

# Path to the dataset zip file
dataset_zip_path = '../datasets/car-4.zip'

# Open the zip file
with zipfile.ZipFile(dataset_zip_path, 'r') as dataset_zip_ref:
    print("Files in", dataset_zip_path, ":")
    for file_name in dataset_zip_ref.namelist():
            if '__MACOSX/' not in file_name:
                print(file_name)
                
    

Files in ../datasets/car-4.zip :
data-desc.txt
test.csv
train.csv


In [3]:
# Open the zip file
with zipfile.ZipFile(dataset_zip_path, 'r') as dataset_zip_ref:
    with dataset_zip_ref.open('data-desc.txt') as desc_file:
        desc_content = desc_file.read().decode("utf-8")
        print(desc_content)
        
    with dataset_zip_ref.open('train.csv') as train_file:
        train_df = pd.read_csv(io.BytesIO(train_file.read()), header=None)
        header = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'evaluation']
        train_df.columns = header
        
    with dataset_zip_ref.open('test.csv') as test_file:
        test_df = pd.read_csv(io.BytesIO(test_file.read()), header=None)
        header = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'evaluation']
        test_df.columns = header

| label values

unacc, acc, good, vgood

| attributes

buying:   vhigh, high, med, low.
maint:    vhigh, high, med, low.
doors:    2, 3, 4, 5more.
persons:  2, 4, more.
lug_boot: small, med, big.
safety:   low, med, high.

| columns
buying,maint,doors,persons,lug_boot,safety,label



In [4]:
train_df.head()

,buying,maint,doors,persons,lug_boot,safety,evaluation
0,low,vhigh,4,4,big,med,acc
1,low,high,5more,4,med,high,vgood
2,vhigh,med,2,2,big,high,unacc
3,high,high,2,2,small,high,unacc
4,vhigh,low,3,2,big,low,unacc


In [5]:
train_df.describe()

,buying,maint,doors,persons,lug_boot,safety,evaluation
count,1000,1000,1000,1000,1000,1000,1000
unique,4,4,4,3,3,3,4
top,vhigh,high,4,4,big,med,unacc
freq,259,255,253,337,341,344,698


In [6]:
for line in desc_content.splitlines()[:-1]:
    if any(attrib in line for attrib in train_df.columns.tolist()[:-1]):
        attrib = line.split(':')[0].strip()
        values = [value.strip() for value in line.split(':')[1].strip()[:-1].split(',')]
        print(attrib, '\n', values, '\n')

buying 
 ['vhigh', 'high', 'med', 'low'] 

maint 
 ['vhigh', 'high', 'med', 'low'] 

doors 
 ['2', '3', '4', '5more'] 

persons 
 ['2', '4', 'more'] 

lug_boot 
 ['small', 'med', 'big'] 

safety 
 ['low', 'med', 'high'] 



### ID3 algorithm

In [7]:
from math import log2

class Dataset:
    
    def __init__(self, train_df, test_df, desc_content) -> None:
        self.train_df = train_df
        self.test_df = test_df
        self.desc_content = desc_content
        
        self.attributes_values_dict = {}
        for line in self.desc_content.splitlines()[:-1]:
            if any(attrib in line for attrib in self.train_df.columns.tolist()[:-1]):
                attrib = line.split(':')[0].strip()
                values = [value.strip() for value in line.split(':')[1].strip()[:-1].split(',')]
                # print(attrib, '\n', values, '\n')
                self.attributes_values_dict[attrib] = values
                
        self.attributes = list(self.attributes_values_dict.keys())
        self.label_column = 'evaluation'
        
        count_down = -1
        for line in self.desc_content.splitlines()[:-1]:
            if 'label values' in line:
                count_down = 2
            count_down -= 1
            if count_down == 0:
                self.label_values = [label_value.strip() for label_value in line.split(',')]


class TreeNode:
    
    def __init__(self, data=None, parent_node=None, child_nodes=None) -> None:
        self.data = data
        self.parent_node = parent_node
        
        # it is a dict of edges as keys and child TreeNode-s as values
        self.child_nodes = child_nodes
        
    def create_child_node(self, edge, child_node):
        # adds child node into this node and returns that child node
        if self.child_nodes == None:
            self.child_nodes = {}
        child_node.parent_node = self
        self.child_nodes[edge] = child_node

class DecisionTreeID3:
    
    def __init__(self, dataset, gain, depth=-1) -> None:
        '''
            train_df: train data's pandas dataframe
            
            depth = -1 means no depth limit set 
            i.e. depth cut off of 0 will never be reached
        '''
        self.dataset = dataset
        self.gain_name = gain
        self.tree = self.id3(self.dataset.train_df, self.dataset.attributes, depth=depth)
    
    def entropy(self, df):
        H = 0
        if len(df) == 0:
            return H
        for label_value in self.dataset.label_values:
            p = len(df[df['evaluation'] == label_value])/len(df)
            if p > 0:
                H -= p*log2(p)
            else:
                H -= 0
        return H
    
    def majority_error(self, df):
        if len(df) == 0:
            return 0
        # ME = 1 - (count of most common label value / total count of labels)
        ME = 1 - (df['evaluation'].value_counts().max()/len(df))
        return ME
    
    def gini_index(self, df):
        GI = 1
        if len(df) == 0:
            return GI
        for label_value in self.dataset.label_values:
            p = len(df[df['evaluation'] == label_value])/len(df)
            GI -= p**2
        return GI
    
    def gain(self, df, attribute, impurity_measure):
        H_s = impurity_measure(df)
        gain = H_s
        for value in self.dataset.attributes_values_dict[attribute]:
            filtered_df = df[df[attribute] == value]
            H_sv = impurity_measure(filtered_df)
            weighted_term = (len(filtered_df)/len(df))*H_sv
            gain -= weighted_term
        return gain
    
    def best_split_attribute(self, train_df, attributes):
        gains_of_attributes = []
        if self.gain_name == 'entropy':
            for attribute in attributes:
                gains_of_attributes.append([attribute, self.gain(train_df, attribute, self.entropy)])
        elif self.gain_name == 'majority_error':
            for attribute in attributes:
                gains_of_attributes.append([attribute, self.gain(train_df, attribute, self.majority_error)])
        elif self.gain_name == 'gini_index':
            for attribute in attributes:
                gains_of_attributes.append([attribute, self.gain(train_df, attribute, self.gini_index)])
        # return the attribute name with the max gain value of all
        return max(gains_of_attributes, key=lambda x: x[1])[0]
    
    def id3(self, train_df, attributes, depth):
        if len(train_df['evaluation'].unique()) == 1:
            label = train_df['evaluation'].unique()[0]
            return TreeNode(label)
        elif attributes == []:
            return TreeNode(data=train_df['evaluation'].value_counts().idxmax())
        elif depth == 0:
            return TreeNode(data=train_df['evaluation'].value_counts().idxmax())
        else:
            root = TreeNode()
            A = self.best_split_attribute(train_df, attributes)
            root.data = A
            for value in self.dataset.attributes_values_dict[A]:
                filtered_df = train_df[train_df[A] == value]
                if len(filtered_df) == 0:
                    child_node = TreeNode(data=train_df['evaluation'].value_counts().idxmax())
                    root.create_child_node(value, child_node=child_node)
                else:
                    new_attributes = attributes.copy()
                    new_attributes.remove(A)
                    child_node = self.id3(filtered_df, new_attributes, depth-1)
                    root.create_child_node(value, child_node=child_node)
            return root
    
    def tree_travel(self, sample_dict, tree):
        if tree.child_nodes == None:
            return tree.data
        else:
            root = tree
            child_node = root.child_nodes[sample_dict[root.data]]
            return self.tree_travel(sample_dict, child_node)
    
    def prediction(self, sample):
        sample_dict = {}
        for index, value in enumerate(sample):
            sample_dict[(self.dataset.attributes+[self.dataset.label_column])[index]] = value
        
        root = self.tree
        return self.tree_travel(sample_dict, root)

### Decision Tree on Dataset

In [8]:
# create Dataset object
dataset = Dataset(train_df, test_df, desc_content)

# creating Decision Tree on `dataset`
dt = DecisionTreeID3(dataset, gain='entropy')

In [9]:
sample = train_df.iloc[2].tolist()

dt.prediction(sample)

'unacc'

In [10]:
count_predict = 0
count_correct_predict = 0
for index in range(len(train_df)):
    sample = train_df.iloc[index].tolist()
    label_predict = dt.prediction(sample)
    count_predict += 1
    if label_predict == sample[-1]:
        count_correct_predict += 1
        
print(f'Average prediction error on Train set = {count_correct_predict*100/count_predict:.2f} %')

Average prediction error on Train set = 100.00 %


In [11]:
count_predict = 0
count_correct_predict = 0
for index in range(len(test_df)):
    sample = test_df.iloc[index].tolist()
    label_predict = dt.prediction(sample)
    count_predict += 1
    if label_predict == sample[-1]:
        count_correct_predict += 1
        
print(f'Average prediction error on Test set = {count_correct_predict*100/count_predict:.2f} %')

Average prediction error on Test set = 60.71 %


In [12]:
# create Dataset object
dataset = Dataset(train_df, test_df, desc_content)

# learning Decision Tree on `dataset`
dt = DecisionTreeID3(dataset, gain='majority_error', depth=-1)

# Train pedictions
count_predict = 0
count_correct_predict = 0
for index in range(len(train_df)):
    sample = train_df.iloc[index].tolist()
    label_predict = dt.prediction(sample)
    count_predict += 1
    if label_predict == sample[-1]:
        count_correct_predict += 1
        
print(f'Average prediction error on Train set = {count_correct_predict*100/count_predict:.2f} %')

# Test predictions
count_predict = 0
count_correct_predict = 0
for index in range(len(test_df)):
    sample = test_df.iloc[index].tolist()
    label_predict = dt.prediction(sample)
    count_predict += 1
    if label_predict == sample[-1]:
        count_correct_predict += 1
        
print(f'Average prediction error on Test set = {count_correct_predict*100/count_predict:.2f} %')

Average prediction error on Train set = 100.00 %
Average prediction error on Test set = 90.66 %


### Q2

(a) [15 points] Implement the ID3 algorithm that supports, information gain, 
majority error and gini index to select attributes for data splits. Besides, your ID3
should allow users to set the maximum tree depth. Note: you do not need to
convert categorical attributes into binary ones and your tree can be wide here.

(b) [10 points] Use your implemented algorithm to learn decision trees from the
training data. Vary the maximum tree depth from 1 to 6 — for each setting,
run your algorithm to learn a decision tree, and use the tree to predict both the
training and test examples. Note that if your tree cannot grow up to 6 levels, you
can stop at the maximum level. Report in a table the average prediction errors
on each dataset when you use information gain, majority error and gini index
heuristics, respectively.

(c) [5 points] What can you conclude by comparing the training errors and the test
errors?

In [33]:
def train_predicts(dt):
    # Train predictions
    count_predict = 0
    count_wrong_predict = 0
    for index in range(len(train_df)):
        sample = train_df.iloc[index].tolist()
        label_predict = dt.prediction(sample)
        count_predict += 1
        if label_predict == sample[-1]:
            count_wrong_predict += 1
    
    # print(f'Average prediction error on Train set = {count_wrong_predict*100/count_predict:.2f} %')
    
    if gain_name == 'entropy':
        errors = acc_using_h
    elif gain_name == 'majority_error':
        errors = acc_using_me
    elif gain_name == 'gini_index':
        errors = acc_using_gi
    
    errors['train'].append(round(count_wrong_predict*100/count_predict,2))


def test_predicts(dt):
    # Test predictions
    count_predict = 0
    count_wrong_predict = 0
    for index in range(len(test_df)):
        sample = test_df.iloc[index].tolist()
        label_predict = dt.prediction(sample)
        count_predict += 1
        if label_predict == sample[-1]:
            count_wrong_predict += 1
    
    # print(f'Average prediction error on Train set = {count_wrong_predict*100/count_predict:.2f} %')
    
    if gain_name == 'entropy':
        errors = acc_using_h
    elif gain_name == 'majority_error':
        errors = acc_using_me
    elif gain_name == 'gini_index':
        errors = acc_using_gi
    
    errors['test'].append(round(count_wrong_predict*100/count_predict,2))


acc_using_h = {'train':[], 'test':[]}
acc_using_me = {'train':[], 'test':[]}
acc_using_gi = {'train':[], 'test':[]}

# create Dataset object
dataset = Dataset(train_df, test_df, desc_content)

for gain_name in ['entropy', 'majority_error', 'gini_index']:
    for depth in range(1,6+1):
        # learning Decision Tree on `dataset`
        dt = DecisionTreeID3(dataset, gain=gain_name, depth=depth)
        
        # Train predictions
        train_predicts(dt)
        
        # Test predictions
        test_predicts(dt)        

In [36]:
print('Prediction Accuracy\n--------------------\n')

print('On training set -\n')
print('Entropy'+10*' '+'-', '   '.join(str(err) for err in acc_using_h['train']))
print('Majority Error'+3*' '+'-', '   '.join(str(err) for err in acc_using_me['train']))
print('Gini Index'+7*' '+'-', '   '.join(str(err) for err in acc_using_gi['train']))

print()

print('On test set -\n')
print('Entropy'+10*' '+'-', '  '.join(str(err) for err in acc_using_h['test']))
print('Majority Error'+3*' '+'-', '  '.join(str(err) for err in acc_using_me['test']))
print('Gini Index'+7*' '+'-', '  '.join(str(err) for err in acc_using_gi['test']))

Prediction Accuracy
--------------------

On training set -

Entropy          - 69.8   69.8   70.0   76.2   83.1   100.0
Majority Error   - 69.8   69.9   81.1   90.3   97.1   100.0
Gini Index       - 69.8   69.9   72.3   81.7   91.1   100.0

On test set -

Entropy          - 70.33  70.33  68.13  64.56  60.71  60.71
Majority Error   - 70.33  68.41  77.61  83.79  90.66  90.66
Gini Index       - 70.33  68.41  71.29  72.53  77.47  77.47
